# Integrate and plot sound, environmental, and climatology data

You don't need to run the installation when on mybinder, this has been taken care of for you

In [ ]:
!pip install rioxarray
!pip install hvplot
!pip install geopandas
!pip install cartopy
!pip install geoviews

In [ ]:
import pathlib
import warnings
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rioxarray
import xarray as xr
from datetime import datetime, timedelta
import hvplot.pandas
from shapely import wkt, geometry
import geopandas as gpd
import minio

In [ ]:
# Supress warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline

# Clear out default notebook settings
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}

# Set figure size and layout
plt.rcParams["figure.figsize"] = [12.00, 5.00]
plt.rcParams["figure.autolayout"] = True

## Set sound and environmental dataset variables

Find variables by exploring https://soundcoop.portal.axds.co.

In [ ]:
erddap_dataset = 'gov-ndbc-44007'
sound_dataset = 'Monh'
max_days = 25
start_date_time = '2021-10-23T20:11:53.211'
end_date_time = '2021-11-18T07:28:18.297'
min_frequency = 21
max_frequency = 24000

In [ ]:
# Update end_date_time if defined temporal range exceeds max_days
time_delta = datetime.fromisoformat(end_date_time) - datetime.fromisoformat(start_date_time)
if time_delta.days > max_days:
    end_date_time = str(datetime.fromisoformat(start_date_time) + timedelta(days=max_days))
    print(f'end_date_time updated to {end_date_time}')

## Download data

In [ ]:
# First we create a function to download data 
def download_data_station(station_name, 
                          client_obj, 
                          bucket_str, 
                          prefix_str, 
                          data_path,
                          name_format,
                          start_datetime, 
                          end_datetime):
    start_datetime_obj = datetime.fromisoformat(start_datetime) 
    end_datetime_obj = datetime.fromisoformat(end_datetime)
    station_folder = pathlib.Path(data_path).joinpath(station_name)
    if not station_folder.exists():
        os.mkdir(station_folder)
    objects = list(client_obj.list_objects(bucket_str, prefix=prefix_str))
    ct = 0
    for i, obj in enumerate(objects):
        object_name = obj.object_name
        path_name = pathlib.Path(object_name).name
        if (not path_name.startswith('.')) & path_name.endswith('.nc'):
            match = re.findall(r"_(\d+)", path_name)[-1]
            file_date = datetime.strptime(match, name_format)
            if (file_date >= start_datetime_obj) & (file_date <= end_datetime_obj):
                download_path = data_path + '/' + station_name + '/' + pathlib.Path(object_name).name
                if os.path.isfile(download_path):
                    print('Already downloaded: ' + download_path)
                else:
                    print('Download ' + str(ct) + ' of ' + str(len(objects)) + ': ' + download_path)
                    object_data = client.get_object(bucket, object_name)
                    if not os.path.isdir(download_path):
                        with open(str(download_path), 'wb') as file_data:
                            for data in object_data:
                                file_data.write(data)
                    file_data.close()
            else: 
                print('Ignored, out of selected period or not a netCDF file ' + path_name)
        ct += 1

In [ ]:
# Define file paths -- can be a string glob or list of explicit paths
local_path = './../data'
sound_paths = local_path + '/' + sound_dataset + '/*.nc'

In [ ]:
# Set up the download for NRS11 data 
client = minio.Minio('storage.googleapis.com')
bucket = 'noaa-passive-bioacoustic'
prefix = 'soundcoop/%s/' % sound_dataset
name_format = '%Y%m%d'
download_data_station(sound_dataset, client, bucket, prefix, local_path, name_format=name_format, start_datetime=start_date_time, end_datetime=end_date_time)

## Read in and clean up sound data

In [ ]:
# Read in data, dropping variables that are sometimes inconsistent across files to 
# prevent open_mfdataset from complaining
sound_ds = xr.open_mfdataset(
    sound_paths,
    engine='netcdf4'
)

# Filter data by defined temporal and frequency ranges
sound_ds = sound_ds.sel(
    time=slice(start_date_time, end_date_time),
    frequency=slice(min_frequency, max_frequency)
)

# Remove data flagged as low quality
minimum_quality = 3
if hasattr(sound_ds, 'quality_flag'):
    sound_ds['psd'] = sound_ds.psd.where(sound_ds.quality_flag <= minimum_quality)

sound_ds

## Download and wrangle environmental data

In [ ]:
erddap_base_url = 'https://erddap.sensors.ioos.us/erddap'

In [ ]:
# Get environmental sensor station lat/lon for use in mapping and querying water temperature climatology data
erddap_metadata_url = f'{erddap_base_url}/info/{erddap_dataset}/index.csv'
env_metadata_df = pd.read_csv(erddap_metadata_url)

env_station_x = env_metadata_df.loc[env_metadata_df['Attribute Name'] == 'geospatial_lon_min']['Value'].item()
env_station_y = env_metadata_df.loc[env_metadata_df['Attribute Name'] == 'geospatial_lat_min']['Value'].item()

wind_speed_units_row = env_metadata_df[
    (env_metadata_df['Row Type'] == 'attribute') & 
    (env_metadata_df['Attribute Name'] == 'units') & 
    (env_metadata_df['Variable Name'] == 'wind_speed')
]
wind_speed_units = wind_speed_units_row.iloc[0]['Value']
wind_speed_units


In [ ]:
wind_speed_to_kts_factors = {
    "m.s-1": 1.94384,
    "mph": 0.86897423357831,
    "kmh": 0.53995555554212126825,
    "ft.s-1": 0.59248243198521155506
}

In [ ]:
if wind_speed_units in wind_speed_to_kts_factors:
    print("Success! Units can be converted from", wind_speed_units,'to','kts')
else:
    print("Error! Wind speed cannot be converted from", wind_speed_units,'to','kts')

In [ ]:
# Grab the same time range covered by the sound data
time_start = np.datetime_as_string(sound_ds.time.min().to_pandas())
time_end = np.datetime_as_string(sound_ds.time.max().to_pandas())

wind_var = 'wind_speed'
swt_var = 'sea_surface_temperature'
wave_var = 'sea_surface_wave_significant_height'
anomaly_var = 'swt_anomaly'
wind_var_kts = 'wind_speed_kts'

erddap_dataset_url = (
    f'{erddap_base_url}/tabledap/{erddap_dataset}.csv'
    f'?time,{wind_var},{swt_var},{wave_var}&time>={time_start}&time<={time_end}'
)

In [ ]:
env_df = pd.read_csv(
    erddap_dataset_url,
    skiprows=[1]  # The second row (index 1) are the column units, which we don't need
)

In [ ]:
# Format the time field and set it as the index
env_df['time'] = pd.to_datetime(env_df['time'])
env_df['wind_speed_kts']=env_df['wind_speed'].apply(lambda x : x*wind_speed_to_kts_factors[wind_speed_units])
env_df = env_df.set_index('time').sort_index()
env_df

In [ ]:
# Save CSV of environmental data
env_df.to_csv(f'env_df_{erddap_dataset}.csv')

## Map station locations

In [ ]:
sound_station_pt = wkt.loads(sound_ds.attrs['geospatial_bounds'])

# Some statoins (e.g. PManan) have their WKT coordinates reversed -- should be x, y (long, lat)
if sound_station_pt.x > 0:
    sound_station_x = sound_station_pt.y
    sound_station_y = sound_station_pt.x
else:
    sound_station_x = sound_station_pt.x
    sound_station_y = sound_station_pt.y


In [ ]:
float(env_station_x)

In [ ]:
stations_gdf = gpd.GeoDataFrame(
    {
        'station': [f'{sound_dataset}', f'{erddap_dataset}'],
        'geometry': [
            geometry.Point(float(sound_station_x), float(sound_station_y)),
            geometry.Point(float(env_station_x), float(env_station_y))
        ]
    },
    crs='epsg:4326'
)

In [ ]:
stations_gdf.hvplot.points(color='station', size=100, geo=True, tiles=True, frame_width=700, frame_height=500)

## Merge sound and environmental data

In [ ]:
# Temporal resolution to which we'll resample the sound and environmental data
# in order to merge them and plot them against each other
temporal_resolution = 'h'

In [ ]:
# Convert env_df to an Xarray Dataset so it can be merged with the sound data
env_ds = env_df.to_xarray()
env_ds['time'] = pd.DatetimeIndex(env_ds['time'].values)

In [ ]:
# Resample sound and environmental data and merge in to a single Xarray Dataset
ds = xr.merge([
    sound_ds.psd.resample(time=temporal_resolution).median(),
    env_ds.resample(time=temporal_resolution).mean()
])
ds

## Calculate and integrate temperature anomaly data

In [ ]:
def get_woa23_temp_at_xy(x, y, month, var='t_mn', depth=0):
    """
    Get 1-degree WOA 2023 temperature values for a given point and month.

    Args:
        x: A longitude value given in decimal degrees
        y: A latitude value given in decimal degrees
        month: The month asn integer from which to extract the value
        var (optional): The temperature variable to use. Defaults to the statistical mean.
        depth (optional): The depth at which to extract the value. Defaults to the surface.
    """
    url = (
        'https://www.ncei.noaa.gov/thredds-ocean/dodsC/woa23/DATA/'
        f'temperature/netcdf/decav/1.00/woa23_decav_t{month:02}_01.nc'
    )
    ds = xr.open_dataset(
        url,
        decode_times=False  # xarray can't handle times defined as "months since ..."
    )

    da = ds.isel(depth=depth)[var]  # Pull out just the variable we're interested in

    # Because nearshore locations are often NaN due to the grid's low resolution
    # we need to interpolate the NaNs to the nearest non-NaN before extracting our value.
    # We use rioxarray to do the interpolations in two dimensions because plain vanilla xarray
    # can only interpolate in one dimension.
    da = da.rio.write_crs(4326)
    da = da.rio.interpolate_na(method='nearest')

    # Then we extract the value, also using the nearest neighbor method because the given
    # x and y values are unlikely to fall exactly on one of the grid's lat/lon coordinate pairs
    val = da.sel(lon=x, lat=y, method='nearest').item()

    return val

In [ ]:
# Define the location of our selected ERDDAP dataset
# Override here if needed
x = env_station_x
y = env_station_y

In [ ]:
url = (
    'https://www.ncei.noaa.gov/thredds-ocean/dodsC/woa23/DATA/'
    f'temperature/netcdf/decav/1.00/woa23_decav_t07_01.nc'
)
da = xr.open_dataset(
    url,
    decode_times=False  # xarray can't handle times defined as "months since ..."
).isel(depth=0)['t_mn']  # Pull out just the variable we're interested in


In [ ]:
da

In [ ]:

# Because nearshore locations are often NaN due to the grid's low resolution
# we need to interpolate the NaNs to the nearest non-NaN before extracting our value.
# We use rioxarray to do the interpolations in two dimensions because plain vanilla xarray
# can only interpolate in one dimension.
da = da.rio.write_crs(4326)
da = da.rio.interpolate_na(method='nearest')
da

In [ ]:
da.coords

In [ ]:
# Then we extract the value, also using the nearest neighbor method because the given
# x and y values are unlikely to fall exactly on one of the grid's lat/lon coordinate pairs
val = da.sel(lon=x, lat=y, method='nearest').item()
val

In [ ]:
ds

In [ ]:
# Assemble a mapping between months and WOA 2023 temperature values
months = list(range(1, 13))
temps = [get_woa23_temp_at_xy(x, y, m) for m in months]
clim_dict = {m: t for m, t in zip(months, temps)}
clim_dict

In [ ]:
# Calculate the sea water temperature anomaly by subtracting the monthly WOA 2023 temperature value
# from each measured sea water temperature value and store it as a new variable
ds[anomaly_var] = ds[swt_var] - [clim_dict[m] for m in ds.time.dt.month.values]
ds

In [ ]:
# Save NetCDF of merged data
ds.to_netcdf(f'merged_data_{sound_dataset}_{erddap_dataset}.nc')

In [ ]:
# Save CSV of merged data
ds.to_dataframe().to_csv(f'merged_data_{sound_dataset}_{erddap_dataset}.csv')

## Plot resampled raw data

In [ ]:
# Plot resampled sound data
ds.psd.plot(x='time', yscale='log')
plt.show()

In [ ]:
ds

In [ ]:
# Plot resampled wind data
ds[wind_var_kts].plot()
plt.show()

In [ ]:
# Plot resampled wave data
ds[wave_var].plot()
plt.show()

In [ ]:
# Plot resampled water temperature data
ds[swt_var].plot()
plt.show()

In [ ]:
# Plot water temperature anomaly data
ds[anomaly_var].plot(linestyle='')
plt.fill_between(ds.time, ds[anomaly_var], 0, where=(ds[anomaly_var] < 0), facecolor='blue', alpha=0.5)
plt.fill_between(ds.time, 0, ds[anomaly_var], where=(ds[anomaly_var] >= 0), facecolor='red', alpha=0.5)
plt.axhline(0, color='k', linestyle='--', linewidth=0.8)
plt.show()

## Plot power spec

In [ ]:
def plot_power_spec(
    ds, 
    conditions=None,
    title=None,
    xlabel='Frequency (hz)', 
    ylabel='Sound pressure level (db)', # use netcdf attribute
    envelope=True
):
    """
    Produces power spec plot given zero or more conditions and labels

    Args:
        ds: An Xarray Dataset consisting of both sound and environmental variables.
        conditions (optional): A single tuple or list of tuples, where each tuple is 
            a condition-label pair, the condition representing a conditional statement to
            be passed to ds.where() and each label representing that condition's legend label.
        title (optional): The plot title.
        xlabel (optional): The X-axis label.
        ylabel (optional): The Y-axis label.
        envelope (optional): Whether or not to plot the 10th to 90th quantile envelope
    """
    fig, ax = plt.subplots()

    # If no conditions were specified, just plot everything
    if conditions is None:
        ds.psd.median(dim='time').plot(x='frequency', xscale='log', ax=ax)

        # Add 10th to 90th quantile envelope
        if envelope:
            plt.fill_between(
                ds.frequency,
                ds.psd.chunk(dict(time=-1)).quantile(0.9, dim='time'),
                ds.psd.chunk(dict(time=-1)).quantile(0.1, dim='time'),
                alpha=0.25
            )

    else:

        # If conditions is a tuple (i.e. a single condition), wrap it in a list so it's iterable
        if isinstance(conditions, tuple):
            conditions = [conditions]

        # Plot each condition in turn
        for c, l in conditions:

            da = ds.where(c, drop=True).psd

            # Only plot if result of where operation has data
            if len(da) > 0:
                da.median(dim='time').plot(x='frequency', xscale='log', ax=ax, label=l)

                # Add 10th to 90th quantile envelope
                if envelope:
                    plt.fill_between(
                        da.frequency,
                        da.chunk(dict(time=-1)).quantile(0.9, dim='time'),
                        da.chunk(dict(time=-1)).quantile(0.1, dim='time'),
                        alpha=0.25
                    )

        ax.legend()
    
    if title is not None:
        ax.set_title(title)

    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    plt.show()


In [ ]:
# Plot everything
plot_power_spec(ds, title='Power spec')

In [ ]:
# Plot wind speed intervals
plot_power_spec(
    ds,
    conditions=[
        (ds[wind_var_kts] <= 10, '< 10 kts'),
        ((ds[wind_var_kts] > 10) & (ds[wind_var] <= 20), '> 10 kts, < 20 kts'),
        (ds[wind_var_kts] > 20, '> 20 kts')
    ],
    title='Power spec by wind speed intervals'
)

In [ ]:
# Plot waves height intervals
plot_power_spec(
    ds,
    conditions=[
        (ds[wave_var] <= 1, '< 1 m'),
        ((ds[wave_var] > 1) & (ds[wave_var] <= 3), '> 1 m, < 3 m'),
        (ds[wave_var] > 3, '> 3 m')
    ],
    title='Power spec by wave height intervals'
)

In [ ]:
# Plot temperature anomaly intervals
plot_power_spec(
    ds,
    conditions=[
        (ds[anomaly_var] < -2, '< -2 °C'),
        ((ds[anomaly_var] < -.5) & (ds[anomaly_var] >= -2), '> -2 °C < -0.5 °C'),
        ((ds[anomaly_var] < .5) & (ds[anomaly_var] >= -.5), '> -0.5 °C < 0.5 °C'),
        ((ds[anomaly_var] >= .5) & (ds[anomaly_var] <= 2), '> 0.5 °C < 2 °C'),
        (ds[anomaly_var] > 2, '> 2 °C')
    ],

    title='Power spec by temperature anomaly intervals'
)